In [24]:
import psycopg2
import os
from dotenv import load_dotenv
import uuid
import pandas as pd

# Load environment variables
load_dotenv()

# Database connection parameters
dbname = os.getenv("DB_NAME")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")

# Path to your EPIs CSV file
csv_file_path = '../data/epis.csv'  # Update to the correct path

# Read CSV file into DataFrame
df_epis = pd.read_csv(csv_file_path)

# Establishing Database Connection
try:
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    cursor = conn.cursor()

    for index, row in df_epis.iterrows():
      # Check and convert to string only if it's not NaN and is a string
      description = row['description'].replace("'", "''") if pd.notna(row['description']) and isinstance(row['description'], str) else ''
      # Apply the same logic to other string fields
      name = row['name'].replace("'", "''") if pd.notna(row['name']) and isinstance(row['name'], str) else ''
      abbreviation = row['abbreviation'].replace("'", "''") if pd.notna(row['abbreviation']) and isinstance(row['abbreviation'], str) else ''
      data_credits_name = row['data_credits_name'].replace("'", "''") if pd.notna(row['data_credits_name']) and isinstance(row['data_credits_name'], str) else ''
      data_credits_url = row['data_credits_url'].replace("'", "''") if pd.notna(row['data_credits_url']) and isinstance(row['data_credits_url'], str) else ''
      

      # Generate UUID for each EPI
      epi_id = uuid.uuid4()

      # Prepare the SQL INSERT statement
      sql_statement = f"""
      INSERT INTO epis (epi_id, name, abbreviation, description, data_credits_name, data_credits_url, created, updated)
      VALUES ('{epi_id}', '{name}', '{abbreviation}', '{description}', '{data_credits_name}', '{data_credits_url}', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP);
      """

      # Execute the SQL statement
      cursor.execute(sql_statement)

    # Commit the transaction
    conn.commit()
    print("All records inserted successfully")

except Exception as e:
    print(f"Error: {e}")

finally:
    # Close the cursor and connection
    if conn:
        cursor.close()
        conn.close()


All records inserted successfully
